In [1]:
from lxml import etree

In [2]:
path = '/media/ilya/My Passport/HSE/VKR/ML_AOI_helper/rawDATA/all-vvts-xml-marchonly/Y1MTB44B5Z05F#MBDX86742001B_T#V510#D-SMT-AOI-41#1#1709274502000#PostRepair#1709276140000.xml'
root = etree.parse(path)
root, root.getroot()

(<lxml.etree._ElementTree at 0x7f989029dc80>,
 <Element {http://tempuri.org/BoardTestXMLExport.xsd}BoardTestXMLExport at 0x7f98902ccec0>)

In [3]:
root.xpath('//BoardTestXMLExport/text', namespaces=root.getroot().nsmap)

[]

In [4]:
root.getroot().tag

'{http://tempuri.org/BoardTestXMLExport.xsd}BoardTestXMLExport'

In [5]:
ns = root.getroot().nsmap
ns

{'ns1': 'http://tempuri.org/BoardTestXMLExport.xsd'}

In [6]:
sn = root.getroot().find("ns1:BoardXML", ns).attrib.get('serialNumber')
sn

'Y1MTB44B5Z05F'

In [7]:
root.getroot().find('BoardXML', namespaces=root.getroot().nsmap)

In [8]:
rootaa = etree.XML("""<ns1:root xmlns:ns1="http://tempuri.org/BoardTestXMLExport.xsd" ><ns1:a x='123'>aText<b/><c/><b/></ns1:a></ns1:root>""")
rootaa

<Element {http://tempuri.org/BoardTestXMLExport.xsd}root at 0x7f98902a9540>

In [9]:
rootaa.find(".//ns1:a", ns).text

'aText'

In [10]:
tests = root.getroot().findall(".//ns1:TestXML", ns) #.attrib.get('name')
len(tests)

74

In [11]:
aa = tests[0].findall(".//ns1:IndictmentXML", ns)
aa

[<Element {http://tempuri.org/BoardTestXMLExport.xsd}IndictmentXML at 0x7f988819c440>,
 <Element {http://tempuri.org/BoardTestXMLExport.xsd}IndictmentXML at 0x7f988819c480>]

In [12]:
aaa = aa[0].find("ns1:RepairActionXML", ns)
aaa

<Element {http://tempuri.org/BoardTestXMLExport.xsd}RepairActionXML at 0x7f988819c5c0>

In [13]:
class Defect:
    assemblyRevision = ''
    serialNumber = ''
    testerName = ''
    type = ''
    indictmentType = ''
    repairOperator = ''
    repairStatus = ''
    designator = ''
    partId = ''
    pinId = ''

    def to_dict(self):
        res = {}
        res['assemblyRevision'] = self.assemblyRevision
        res['serialNumber'] = self.serialNumber
        res['testerName'] = self.testerName
        res['type'] = self.type
        res['indictmentType'] = self.indictmentType
        res['repairOperator'] = self.repairOperator
        res['repairStatus'] = self.repairStatus
        res['designator'] = self.designator
        res['partId'] = self.partId
        res['pinId'] = self.pinId

        return res

In [14]:
def parse_file_for_defects(path):
    root = etree.parse(path)
    ns = root.getroot().nsmap
    res = [] #Defect()
    serialNumber = root.getroot().find("ns1:BoardXML", ns).attrib.get('serialNumber')
    testerName = root.getroot().find("ns1:StationXML", ns).attrib.get('testerName')
    assemblyRevision = root.getroot().find("ns1:BoardXML", ns).attrib.get('assemblyRevision')

    # TestXML = root.getroot().findall(".//ns1:TestXML", ns)
    TestXML = root.getroot().findall(".//ns1:TestXML", ns)
    for test in TestXML:
        name = test.attrib.get('name')
        IndictmentXML = test.findall(".//ns1:IndictmentXML", ns)
        PinXML = test.findall(".//ns1:PinXML", ns)
    
        for Indictment in IndictmentXML:
            tmp = Defect()
            tmp.assemblyRevision = assemblyRevision
            tmp.serialNumber = serialNumber
            tmp.testerName = testerName
            tmp.designator = name
            tmp.type = 'IndictmentXML'
            tmp.indictmentType = Indictment.attrib.get('indictmentType')
            tmp.repairOperator = Indictment.find("ns1:RepairActionXML", ns).attrib.get('repairOperator')
            tmp.repairStatus = Indictment.find("ns1:RepairActionXML", ns).attrib.get('repairStatus')
            tmp.partId = Indictment.find("ns1:ComponentXML", ns).attrib.get('partId')
            res.append(tmp)

        partId = IndictmentXML[0].find("ns1:ComponentXML", ns).attrib.get('partId')
        for pin in PinXML:
            repairStatus = pin.attrib.get('repairStatus')
            id = pin.attrib.get('id')
            PinIndictmentXML = pin.findall(".//ns1:PinIndictmentXML", ns)

            for pinIndictment in PinIndictmentXML:
                tmp = Defect()
                tmp.assemblyRevision = assemblyRevision
                tmp.serialNumber = serialNumber
                tmp.testerName = testerName
                tmp.designator = name
                tmp.type = 'PinXML'
                tmp.indictmentType = Indictment.attrib.get('indictmentType')
                tmp.repairOperator = Indictment.find("ns1:RepairActionXML", ns).attrib.get('repairOperator')
                tmp.repairStatus = repairStatus
                tmp.partId = partId
                tmp.pinId = id
                res.append(tmp)
        
    return res


In [15]:
file_1 = parse_file_for_defects(path)
len(file_1)

348

In [16]:
file_1[200].to_dict()

{'assemblyRevision': 'MBDX86742001B_T',
 'serialNumber': 'Y1MTB44B5Z05F',
 'testerName': 'D-SMT-AOI-41',
 'type': 'PinXML',
 'indictmentType': 'Coplanarity',
 'repairOperator': 'Svetlana_VVTS08',
 'repairStatus': 'False Call',
 'designator': '1:c22',
 'partId': 'GRM188C80G106KE47D',
 'pinId': '0'}

In [17]:
from os import listdir
mypath = '/media/ilya/My Passport/HSE/VKR/ML_AOI_helper/rawDATA/all-vvts-xml-marchonly/'
all_files = listdir(mypath)

In [18]:
all_files[4]

'Y2G9A44BYF00D#MBDX86691004A_T#V510#D-SMT-AOI-51#1#1709316486000#PostRepair#1709317954000.xml'

In [27]:
all_defects = []

for file in all_files:
    tmp = parse_file_for_defects(mypath + file)
    for t in tmp:
        all_defects.append(t.to_dict())

In [28]:
len(all_defects)

354537

In [29]:
all_defects[200002]

{'assemblyRevision': 'MBDX86742001B_T',
 'serialNumber': 'Y1MTB44BYF01E',
 'testerName': 'D-SMT-AOI-41',
 'type': 'PinXML',
 'indictmentType': 'Polarity',
 'repairOperator': 'Larisa_VVTS08',
 'repairStatus': 'False Call',
 'designator': '1:d91',
 'partId': 'QTLP601CRTR',
 'pinId': '0'}

In [30]:
import pandas as pd

df = pd.DataFrame(all_defects)

In [32]:
df[df.repairStatus != 'False Call'].shape[0]

4618

In [34]:
df[df.repairStatus == 'False Call'].shape[0] + df[df.repairStatus != 'False Call'].shape[0]

354537

In [36]:
df[df.repairStatus != 'False Call'].head

<bound method NDFrame.head of        assemblyRevision   serialNumber    testerName           type  \
49      MBDX86691004A_T  Y2G9A44BYF00B  D-SMT-AOI-51  IndictmentXML   
50      MBDX86691004A_T  Y2G9A44BYF00B  D-SMT-AOI-51         PinXML   
51      MBDX86691004A_T  Y2G9A44BYF00B  D-SMT-AOI-51         PinXML   
52      MBDX86691004A_T  Y2G9A44BYF00B  D-SMT-AOI-51  IndictmentXML   
53      MBDX86691004A_T  Y2G9A44BYF00B  D-SMT-AOI-51         PinXML   
...                 ...            ...           ...            ...   
354090  MBDX86691004A_T  Y2G9A44BYF008  D-SMT-AOI-51         PinXML   
354091  MBDX86691004A_T  Y2G9A44BYF008  D-SMT-AOI-51         PinXML   
354187  MBDX86691004A_T  Y2G9A44BYF009  D-SMT-AOI-51  IndictmentXML   
354188  MBDX86691004A_T  Y2G9A44BYF009  D-SMT-AOI-51         PinXML   
354189  MBDX86691004A_T  Y2G9A44BYF009  D-SMT-AOI-51         PinXML   

       indictmentType repairOperator repairStatus designator  \
49        Coplanarity        Semen_O     Repaired    

In [37]:
from IPython.display import display, HTML

display(df[df.repairStatus != 'False Call'])

,assemblyRevision,serialNumber,testerName,type,indictmentType,repairOperator,repairStatus,designator,partId,pinId
49,MBDX86691004A_T,Y2G9A44BYF00B,D-SMT-AOI-51,IndictmentXML,Coplanarity,Semen_O,Repaired,1:rq238,01_1_01_2220301-Y01,
50,MBDX86691004A_T,Y2G9A44BYF00B,D-SMT-AOI-51,PinXML,Coplanarity,Semen_O,Repaired,1:rq238,01_1_01_2220301-Y01,0
51,MBDX86691004A_T,Y2G9A44BYF00B,D-SMT-AOI-51,PinXML,Coplanarity,Semen_O,Repaired,1:rq238,01_1_01_2220301-Y01,1
52,MBDX86691004A_T,Y2G9A44BYF00B,D-SMT-AOI-51,IndictmentXML,Coplanarity,Semen_O,Repaired,1:rp1186,01_1_01_2000001-Y01,
53,MBDX86691004A_T,Y2G9A44BYF00B,D-SMT-AOI-51,PinXML,Coplanarity,Semen_O,Repaired,1:rp1186,01_1_01_2000001-Y01,0
...,...,...,...,...,...,...,...,...,...,...
354090,MBDX86691004A_T,Y2G9A44BYF008,D-SMT-AOI-51,PinXML,XS paste,Semen_O,Repaired,1:rs455,01_1_01_0075301-Y01,0
354091,MBDX86691004A_T,Y2G9A44BYF008,D-SMT-AOI-51,PinXML,XS paste,Semen_O,Repaired,1:rs455,01_1_01_0075301-Y01,1
354187,MBDX86691004A_T,Y2G9A44BYF009,D-SMT-AOI-51,IndictmentXML,BadJoint,Semen_O,Repaired,1:rp816,10_03_01_112011-01Y,
354188,MBDX86691004A_T,Y2G9A44BYF009,D-SMT-AOI-51,PinXML,BadJoint,Semen_O,Repaired,1:rp816,10_03_01_112011-01Y,0


In [38]:
df[df.repairStatus != 'False Call'].describe()

,assemblyRevision,serialNumber,testerName,type,indictmentType,repairOperator,repairStatus,designator,partId,pinId
count,4618,4618,4618,4618,4618,4618,4618,4618,2741,4618
unique,13,426,5,2,22,11,1,366,61,62
top,MBDX8674X001B,Y1MTB44B5Z04J,D-SMT-AOI-41,IndictmentXML,Lifted lead,FateevD,Repaired,top:1:U209:7,MEC1-108-02-L-D-RA1-SL,
freq,1514,205,1813,2351,631,1221,4618,288,627,2351
